In [102]:
import numpy as np
import time
from scipy.optimize import ridder

SOL_STRING = "A solution was found in {0} iterations with a-b = {1}, f(c) = {2}"

def timeit(func):
    def wrapper(*arg, **kw):
        t1 = time.perf_counter()
        res = func(*arg, **kw)
        t2 = time.perf_counter()
        print("{0} took {1}s".format(func.__name__, (t2 - t1)))
        return res
    return wrapper

arr = np.genfromtxt("xdata.txt")

mu_hat = arr.mean()


def funderivativel(lambdas, mu, arr):
    n = len(arr)
    log_likelihoods_derivatives = (n/(2*lambdas)) - (((arr - mu)**2)/((2*mu**2)*arr)).sum()
    return log_likelihoods_derivatives

In [103]:
@timeit
def rootfindscipy(f, a, b, args):
    return ridder(f, a, b,  args, full_output=True)

rootfindscipy(funderivativel, 0.1, 3, (mu_hat, arr))

rootfindscipy took 0.0012621000059880316s


(1.477439027305124,
       converged: True
            flag: converged
  function_calls: 14
      iterations: 6
            root: 1.477439027305124)

In [104]:
@timeit
def bisection(f, a, b, args, N_MAX=1000, DELTA=0.001):
    if f(a, *args)*f(b, *args) > 0:
        raise Exception("f(a)f(b) > 0, This function may not converge")

    n = 0
    while n < N_MAX:
        c = (b + a) / 2
        f_c = f(c, *args)
        
        if f_c == 0 or abs(c) < DELTA:
            print(SOL_STRING.format(n, b-a, f_c))
            return c
        n += 1

        if f_c*f(a, *args) > 0:
            a = c
        else:
            b = c
        
    raise Exception("No solution could be found")

bisection(funderivativel, 0.1, 3, (mu_hat, arr))

A solution was found in 53 iterations with a-b = 4.440892098500626e-16, f(c) = 0.0
bisection took 0.004286899988073856s


1.4774390273041236

In [105]:
@timeit
def falsepos(f, a, b, args, N_MAX=1000, DELTA=0.001):
    if f(a, *args)*f(b, *args) > 0:
        raise Exception("f(a)f(b) > 0, This function may not converge")

    n = 0
    while n < N_MAX:
        f_a = f(a, *args)
        f_b = f(b, *args)

        c = (a*f_b - b*f_a) / (f_b - f_a)
        f_c = f(c, *args)

        if f_c == 0 or abs(f_c)<DELTA:
            print(SOL_STRING.format(n, b-a, f_c))
            return c

        if f_a * f_c < 0:
            b = c
        else:
            a = c
        
        n += 1
        
    raise Exception("No solution could be found")

falsepos(funderivativel, 0.1, 3, (mu_hat, arr), N_MAX=10000)

A solution was found in 182 iterations with a-b = 1.3774434227947603, f(c) = -0.0009386849221755256
falsepos took 0.009076899994397536s


1.477443125287347

In [106]:
@timeit
def modfalsepos(f, a, b, args, N_MAX=1000, DELTA=0.001):
    if f(a, *args)*f(b, *args) > 0:
        raise Exception("f(a)f(b) > 0, This function may not converge")

    n = 0
    while n < N_MAX:
        f_a = f(a, *args)
        f_b = f(b, *args)

        if f_a*f_b < 0:
            c = (a*f_b - 2*b*f_a) / (f_b - 2*f_a)
        else:
            c = (2*a*f_b - b*f_a) / (2*f_b - f_a)
        f_c = f(c, *args)

        if f_c == 0 or abs(f_c)<DELTA:
            print(SOL_STRING.format(n, b-a, f_c))
            return c

        if f_a * f_c < 0:
            b = c
        else:
            a = c
        
        n += 1
        
    raise Exception("No solution could be found")

modfalsepos(funderivativel, 0.1, 3, (mu_hat, arr))

A solution was found in 369 iterations with a-b = 1.3774435356494554, f(c) = -0.0009977339789770667
modfalsepos took 0.01934510000864975s


1.4774433830764644